In [6]:
import os
from os.path import join
import numpy as np
import torch
import random
import math
# from sklearn.model_selection import KFold
# from transformers import AutoModelForCausalLM, TrainingArguments, Trainer, DataCollatorForLanguageModeling, set_seed
# from huggingface_hub import login
# import wandb
# from env_config import hf_api_key_w, data_path, hf_usn, wandb_project, seedn
seedn=42
data_path=os.environ['DATA_PATH']
from utils.data_config import separator_dict, gpt_model_dict, arrow_l2r
from utils.data import load_and_sample_data
# from utils.model_utils import setup_tokenizer, tokenize_dataset, get_epoch_lists
from utils.utilities import make_dict
random.seed(seedn)
task='tgt2ceq'
sample_ratio = 1
arrow = arrow_l2r 
separator, cut = separator_dict[task], ';'
dataset = load_and_sample_data(data_path, task, separator, te_ratio=0.1, cut=cut, arrow=arrow, sample_ratio=sample_ratio, save_idx_name=None)

from pprint import pprint
from collections import Counter
for rxnoi in ['CaTi4Cu3O12 ||',]:
    for split in ['test','train']:
        filtered_dataset = dataset[split].filter(lambda example: example['label'] == rxnoi)
        if split=='test' and len(filtered_dataset)==0:
            break
        print(split, "set, size of", len(filtered_dataset))
        pprint(Counter([example['text'] for example in filtered_dataset]).most_common())
        print()

Filter: 100%|██████████| 1948/1948 [00:00<00:00, 131392.39 examples/s]


test set, size of 11
[('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4O12 + 1 CO2', 10),
 ('CaTi4Cu3O12 || 4 TiO2 + 3 CuO + 1 Ca(OH)2 -> 1 CaCu3Ti4O12 + 1 H2O', 1)]



Filter: 100%|██████████| 17540/17540 [00:00<00:00, 160630.07 examples/s]

train set, size of 77
[('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4O12 + 1 CO2', 54),
 ('CaTi4Cu3O12 || 3 Cu(NO3)2·3H2O + 4 TiO2 + 1 Ca(NO3)2·4H2O -> 1 CaCu3Ti4O12 '
  '+ 13 H2O + 8 NO2 + 2 O2',
  3),
 ('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 Ca(OH)2 -> 1 CaCu3Ti4O12 + 1 H2O', 2),
 ('CaTi4Cu3O12 || 1 CaO + 4 TiO2 + 3 CuO -> 1 CaCu3Ti4O12', 2),
 ('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4.0O12 + 1 CO2', 1),
 ('CaTi4Cu3O12 || 3 CuO + 4 TiO2 + 1 CaCO3 -> 1 CaCu3Ti4O12 + 1 CO2 ', 1),
 ('CaTi4Cu3O12 || 3 CuO + 1 CaCO3 + 4 anatase-TiO2 -> 1 CaCu3Ti4O12 + 1 CO2',
  1),
 ('CaTi4Cu3O12 || 3 CuO() + 1 CaCO3 + 4 TiO2() -> 1 CaCu3Ti4O12 + 1 CO2', 1),
 ('CaTi4Cu3O12 || 3 CuO + 1 CaCO3 + 4 TiO2 -> 1 CaCu3Ti4O12 + 1 CO2', 1),
 ('CaTi4Cu3O12 || 1 CaO + 3 CuO + 4 TiO2 -> 1 CaCu3Ti4O12', 1),
 ('CaTi4Cu3O12 || 3 Cu(NO3)2·3H2O + 1 Ca(NO3)2·4H2O + 4 C16H36O14Ti + 74 O2 -> '
  '1 CaCu3Ti4O12 + 64 CO2 + 85 H2O + 8 NO2',
  1),
 ('CaTi4Cu3O12 || 4 C12H28O4Ti + 3 Cu(NO3)2 + 1 Ca(NO3)2·4H2O + 7